In [3]:
from data_common.notebook import *

# Petitions analysis

Quick notebook to examine the results of the significance detection.

In [19]:
petitions = pd.read_csv(Path("data", "interim", "petitions.csv"))

# reduce to just the petitions with more than 1000 signatures classed and where is_environmental is true
petitions = petitions[
    (petitions["signature_count"] > 1000) & (petitions["is_environmental"] == True)
]
petitions = petitions.sort_values("signature_count", ascending=False)

In [24]:
df = pd.read_csv(
    Path("data", "interim", "constituency_signatures_with_significance.csv")
)

# limit to just the petition_id that are present in the petitions dataframe
df = df[df["petition_id"].isin(petitions["id"].tolist())]

# reduce to the top ten petitions per constituency (gss)
df = df.sort_values("signatures", ascending=False).groupby("gss").head(10)
df

# for each constituency, calculate the total number of signatures, and how many of those are significant

# first, group by constituency and sum the signatures


def get_sum_signatures(df):
    count = df["signatures"].sum()
    sig = df["significant"].sum()
    total = len(df)
    return pd.Series({"count": count, "sig": sig, "qualifying_petitions": total})


group = df.groupby("gss").apply(get_sum_signatures).reset_index()

,gss,count,sig,qualifying_petitions
0,E14000530,2421.0,8.0,10.0
1,E14000531,1545.0,4.0,10.0
2,E14000532,2148.0,3.0,10.0
3,E14000533,2524.0,8.0,10.0
4,E14000534,3889.0,9.0,10.0
...,...,...,...,...
645,W07000076,1716.0,2.0,10.0
646,W07000077,1340.0,3.0,10.0
647,W07000078,2331.0,5.0,10.0
648,W07000079,2277.0,7.0,10.0


# What are the statistics on how the total number of signatures (in top ten) are distributed

Average of around 2,000 signatures, some much lower, some higher!

In [21]:
group["count"].describe()

count     650.00
mean     2236.36
std       993.23
min       275.00
25%      1581.75
50%      2121.00
75%      2787.75
max      9303.00
Name: count, dtype: float64

# What are the statistics on if the total signatures in top ten are significant

On average 6ish out of top ten are significant for the constituency - meaning they standout from all constituencies (if not necessarily *similar* constituencies).

In [18]:
group["sig"].describe()

count    650.00
mean       6.46
std        1.97
min        1.00
25%        5.00
50%        6.00
75%        8.00
max       10.00
Name: sig, dtype: float64

In [22]:
len(df)

6500